### Download the corpora and split it in training and test sets, structuring a dataframe.




In [1]:
import urllib.request  #  download files
import tarfile  #  unzip files
import shutil
import os
import urllib.request  #  download files
import zipfile  #  unzip files


**Splits**: documents 1-100 are the train set, 101-150 validation set, 151-199 test set.

**Splitting**: you can decide to split documents into sentences or not, the choice is yours.


**I/O structure**: The input data will have three dimensions: 1-documents/sentences, 2-token, 3-features; for the output there are 2 possibilities: if you use one-hot encoding it will be 1-documents/sentences, 2-token labels, 3-classes, if you use a single integer that indicates the number of the class it will be 1-documents/sentences, 2-token labels.

In [2]:
dataset_folder = os.path.join(os.getcwd(), "Dataset")

if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)

url = "https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip"

dataset_path = os.path.join(dataset_folder, "dependency_treebank.zip")

if not os.path.exists(dataset_path):
    urllib.request.urlretrieve(url, dataset_path)
    print("Successful download")

with zipfile.ZipFile(dataset_path,"r") as zip_ref:
    zip_ref.extractall("Dataset")

print("Successful extraction")

Successful extraction


In [3]:
data_folder=os.getcwd()+"/Dataset/dependency_treebank/"

In [4]:
# Just needed in case you'd like to append it to an array
data = []

for filename in os.listdir(data_folder):
    if filename.endswith("dp"): 
        data.append(filename)

Cicla su una frase
- Fintanto che non sei arrivato ad un punto seguito da uno spazio:
    * Leggi tutti i caratteri di una parola fino a che non trovi un spazio
    * Salta a riga dopo e ripeti


1. Trova indice righe splittate in train
2. Trova indice righe splittate in test
3. Trova indice righe splittate in validation


Commento

In [5]:
import re

Voglio tutti i file name tali che:
1. Per il **train** devono iniziare con 00 o essere 0100
2. Per il **val** devono iniziare con 01 piu (0..4) oppure essere 0150
3. Per il **test** devono essere 015(1..9) oppure tutti quelli che iniziano che vanno da 01(6..9)(0..9)



In [6]:
# CODICE ORIGINALE (VEDERE CELLA SEGUENTE PER MODIFICHE PROPOSTE) 
sentences = []
words = []
unique_tags_train = []
unique_tags_test = []
unique_tags_val = []
tags = []
tag2 = []
file_len = []
cnt_sents = 0
docs = sorted(os.listdir(data_folder))
for filename in docs:
    if filename.endswith(".dp"): 
        with open(os.path.join(data_folder, filename), mode='r', encoding='utf-8') as text:
                for i,line in enumerate(text):
                    if len(line.split())>0:
                        word, tag, _ = line.split("\t")    # prova a inventare un altro modo split
                        words.append(word)
                        tags.append(tag)

                        #regex train

                        #regex val

                        #regex test
                        if tag not in unique_tags_train  and re.match("wsj([00]).*", filename) or filename == "wsj_100.dp":
                            unique_tags_train.append(tag)
                        elif tag not in unique_tags_val  and filename.startswith("wsj_01") and not filename.startswith("wsj_151.dp"):
                             unique_tags_val.append(tag)
                        elif tag not in unique_tags_test:
                            unique_tags_test.append(tag)
                        if word == ".": # condizione fine frase
                            sentences.append(words[:i])
                            cnt_sents+=1
                            tag2.append(tags[:i])
                            words = []
                            tags =[]
                file_len.append(cnt_sents)
        cnt_sents = 0


In [7]:
# SB MODIFIED CODE - START
sentences = []
words = []
unique_tags_train = []
unique_tags_test = []
unique_tags_val = []
tags = []
tag2 = []
file_len = []
file_len_sent = []
file_len_tags = []

cnt_words_train = 0
cnt_words_val = 0
cnt_words_test = 0
cnt_words_sent = 0
cnt_tags_sent = 0
cnt_sents = 0
cnt_sents_train = 0
cnt_sents_val = 0
cnt_sents_test = 0

docs = sorted(os.listdir(data_folder))
for filename in docs:
    if filename.endswith(".dp"): 
        with open(os.path.join(data_folder, filename), mode='r', encoding='utf-8') as text:
                for i,line in enumerate(text):
                    if len(line.split())>0:
                        word, tag, _ = line.split("\t")    # prova a inventare un altro modo split
                        words.append(word)
                        tags.append(tag)
                        cnt_words_sent += 1                        
                        cnt_tags_sent += 1
                        #regex train

                        #regex val
                        
                        #regex test
                        #if tag not in unique_tags_train  and re.match("wsj([00]).*", filename) or filename == "wsj_100.dp":
                        if (filename.startswith("wsj_00") or filename == "wsj_0100.dp"):
                            cnt_words_train +=1
                            if (tag not in unique_tags_train): 
                                unique_tags_train.append(tag)
                            if word == ".": # condizione fine frase
                                sentences.append(words[:i])
                                cnt_sents += 1
                                cnt_sents_train+=1
                                tag2.append(tags[:i])
                                words = []
                                tags =[]
                                file_len_tags.append(cnt_tags_sent)
                                file_len_sent.append(cnt_words_sent)
                                cnt_words_sent = 0
                                cnt_tags_sent = 0

                        elif (not filename == "wsj_0100.dp") and (filename.startswith("wsj_010") or filename.startswith("wsj_011") or filename.startswith("wsj_012") or filename.startswith("wsj_013") or filename.startswith("wsj_014") or filename.startswith("wsj_0150")):
                            cnt_words_val +=1
                            if (tag not in unique_tags_val):
                                unique_tags_val.append(tag)
                            if word == ".": # condizione fine frase
                                sentences.append(words[:i])
                                cnt_sents += 1
                                cnt_sents_val+=1
                                tag2.append(tags[:i])
                                words = []
                                tags =[]
                                file_len_tags.append(cnt_tags_sent)
                                file_len_sent.append(cnt_words_sent)
                                cnt_words_sent = 0
                                cnt_tags_sent = 0

                        elif (not filename == "wsj_0150.dp") and (filename.startswith("wsj_015") or filename.startswith("wsj_016") or filename.startswith("wsj_017") or filename.startswith("wsj_018") or filename.startswith("wsj_019")):
                            cnt_words_test +=1
                            if (tag not in unique_tags_test):
                                unique_tags_test.append(tag)
                            if word == ".": # condizione fine frase
                                sentences.append(words[:i])
                                cnt_sents += 1
                                cnt_sents_test+=1
                                tag2.append(tags[:i])
                                words = []
                                tags =[]
                                file_len_tags.append(cnt_tags_sent)
                                file_len_sent.append(cnt_words_sent)
                                cnt_words_sent = 0
                                cnt_tags_sent = 0

                file_len.append(cnt_sents)
        cnt_sents = 0


* Trova lunghezza righe documento
* a ultima sentence aggiungi un marker per indicare che è end of file 
* fino a quella sentence saranno doc = nome doc

In [8]:
print(f"train: {sorted(unique_tags_train)} \n")
print(f"validation: {sorted(unique_tags_val)} \n")
print(f"test: {sorted(unique_tags_test)}")


train: ['#', '$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``'] 

validation: ['#', '$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``'] 

test: ['$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'CC', 'CD', 'DT', 'EX', 'IN', 'JJ', 'JJR', 'JJS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


In [9]:
print(f"train: {len(unique_tags_train)} \n")
print(f"validation: {len(unique_tags_val)} \n")
print(f"test: {len(unique_tags_test)}")

train: 45 

validation: 44 

test: 40


In [10]:
print(list(set(unique_tags_train).difference(set(unique_tags_val))))
print(list(set(unique_tags_train).difference(set(unique_tags_test))))
print(list(set(unique_tags_val).difference(set(unique_tags_test))))

['SYM']
['UH', '#', 'FW', 'LS', 'SYM']
['#', 'UH', 'FW', 'LS']


In [11]:
# SB Added Cell just to count sentences per each dataset
print(cnt_sents_train)
print(cnt_sents_val)
print(cnt_sents_test)


1936
1256
636


In [12]:
file_len[:5]

[2, 1, 30, 17, 3]

In [13]:
import pandas as pd
import numpy as np

In [14]:
# create single dataframe row
dataframe_rows = {
#    'docs': [doc for doc in docs],
    'sentences': sentences,
    'tags': tag2,
    "docs": np.repeat(docs, file_len)
}


In [15]:
df = pd.DataFrame(dataframe_rows)
df.head(10)

,sentences,tags,docs
0,"[Pierre, Vinken, ,, 61, years, old, ,, will, j...","[NNP, NNP, ,, CD, NNS, JJ, ,, MD, VB, DT, NN, ...",wsj_0001.dp
1,"[Mr., Vinken, is, chairman, of, Elsevier, N.V....","[NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, NNP, ...",wsj_0001.dp
2,"[Rudolph, Agnew, ,, 55, years, old, and, forme...","[NNP, NNP, ,, CD, NNS, JJ, CC, JJ, NN, IN, NNP...",wsj_0002.dp
3,"[A, form, of, asbestos, once, used, to, make, ...","[DT, NN, IN, NN, RB, VBN, TO, VB, NNP, NN, NNS...",wsj_0003.dp
4,"[The, asbestos, fiber, ,, crocidolite, ,, is, ...","[DT, NN, NN, ,, NN, ,, VBZ, RB, JJ, IN, PRP, V...",wsj_0003.dp
5,"[Lorillard, Inc., ,, the, unit, of, New, York-...","[NNP, NNP, ,, DT, NN, IN, JJ, JJ, NNP, NNP, WD...",wsj_0003.dp
6,"[Although, preliminary, findings, were, report...","[IN, JJ, NNS, VBD, VBN, RBR, IN, DT, NN, IN, ,...",wsj_0003.dp
7,"[A, Lorillard, spokewoman, said, ,, ``, This, ...","[DT, NNP, NN, VBD, ,, ``, DT, VBZ, DT, JJ, NN, .]",wsj_0003.dp
8,"[We, 're, talking, about, years, ago, before, ...","[PRP, VBP, VBG, IN, NNS, IN, IN, NN, VBD, IN, ...",wsj_0003.dp
9,"[There, is, no, asbestos, in, our, products, n...","[EX, VBZ, DT, NN, IN, PRP$, NNS, RB, .]",wsj_0003.dp


In [16]:
print('Dataset Info:')
print(df.info(),'\n')

print('Dataset Head:')
display(df.head())

print('\nDataset Description:')
display(df.describe())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3828 entries, 0 to 3827
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentences  3828 non-null   object
 1   tags       3828 non-null   object
 2   docs       3828 non-null   object
dtypes: object(3)
memory usage: 89.8+ KB
None 

Dataset Head:


,sentences,tags,docs
0,"[Pierre, Vinken, ,, 61, years, old, ,, will, j...","[NNP, NNP, ,, CD, NNS, JJ, ,, MD, VB, DT, NN, ...",wsj_0001.dp
1,"[Mr., Vinken, is, chairman, of, Elsevier, N.V....","[NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, NNP, ...",wsj_0001.dp
2,"[Rudolph, Agnew, ,, 55, years, old, and, forme...","[NNP, NNP, ,, CD, NNS, JJ, CC, JJ, NN, IN, NNP...",wsj_0002.dp
3,"[A, form, of, asbestos, once, used, to, make, ...","[DT, NN, IN, NN, RB, VBN, TO, VB, NNP, NN, NNS...",wsj_0003.dp
4,"[The, asbestos, fiber, ,, crocidolite, ,, is, ...","[DT, NN, NN, ,, NN, ,, VBZ, RB, JJ, IN, PRP, V...",wsj_0003.dp



Dataset Description:


,sentences,tags,docs
count,3828,3828,3828
unique,3819,3795,199
top,"[Terms, were, n't, disclosed, .]","[NNS, VBD, RB, VBN, .]",wsj_0118.dp
freq,8,8,172


In [17]:
# SB - START NEW CODE SECTION

import torchtext

# Dowload GloVe Vocabulary 6B (It includes sources as Gigaword ( newswire ) consistent with the domain of the Input data (WSJ articles))
V1 = torchtext.vocab.GloVe(name='6B', dim=50)

In [18]:
# SB - CONTINUE NEW CODE SECTION
# One hot encoding: the task to the performed by the model is a multi-class classification. 
# Labels are not encoded assigning integer values to differnt classes, but rather by one hot encoding.

# arrays dimensions initialization 
embedding_dim = 50 # set accordingly to the embedding chosen (GloVe 6B 50d)
missing_pos_tag = 1 # assuming to add a single one hot encoding for any POS tag not present in training
np.random.seed(42) # setting seed to obtain repeatable random embeddings

# dimensions initialization 
tot_sentence_num = len(df['sentences']) # Total number of sentences
tot_sentence_train = cnt_words_train # Total number of training sentences
tot_sentence_val = cnt_words_val # Total number of validation sentences
tot_sentence_test = cnt_words_test # Total number of test sentences

tot_token_num = max(file_len_tags) # Total number of tokens
max_sentence_len = max(file_len_sent) # Max sentence length (number of tokens)

# One Hot Encoding of POS Tags
ohe_df = pd.get_dummies(unique_tags_train) # Get one hot encoding for POS tags in the training dataset
ohe_df['UKN'] = [0] * len(unique_tags_train) # Add a column for 'UKN' POS tags

ohe = np.zeros((len(unique_tags_train)+1,len(unique_tags_train)+1)) # Create array of same size as encoding from training plus 1 row and 1 column for UKN
for i in range(0,len(ohe_df)):
  ohe[i] = ohe_df.iloc[i] # copy one hot encoding
ohe[(len(unique_tags_train),len(unique_tags_train))] = 1 # Add the encoding for UKN POS tag

# Numpy arrays X (token embddings) and Y (One Hot Encoding of POS tag classes) initialization:
# X shape: dim 0 (total sentences per dataset), dim 1 (max tokens), dim 2 (features (embedding vectors)) 
X_train = np.zeros((cnt_sents_train,max_sentence_len,embedding_dim))
X_val = np.zeros((cnt_sents_val,max_sentence_len,embedding_dim))
X_test = np.zeros((cnt_sents_test,max_sentence_len,embedding_dim))

# Y shape: dim 0 (total sentences per dataset), dim 1 (max tokens), dim 2 (One Hot Encoding vectors)) 
Y_train = np.zeros((cnt_sents_train,max_sentence_len,ohe.shape[1]))
Y_val = np.zeros((cnt_sents_val,max_sentence_len,ohe.shape[1]))
Y_test = np.zeros((cnt_sents_test,max_sentence_len,ohe.shape[1]))

# Out of Vocabulary initialization
OOV1 = {}
OOV2 = {}
OOV3 = {}

# X and Y arrays update retriving token/tag pairs from df dataframe and retriving embedding and one hot encoding vectors
for index,row in df.iterrows(): # Iterate over the sentences in df dataframe and update X with the token embedding vector and Y with the class one hot encoding

    # Training
    # The model will be trained on X_train and Y_train containing V1 and OOV1 embeddings (V2) and one hot encodings found in training
    if ((row['docs'].startswith("wsj_00")) or (row['docs'] == "wsj_0100.dp")):
      for i in range(0,len(row['sentences'])): # Iterate over the words of the fetched sentence 

        # Embedding from V1 (GloVe Voc. 6B), if not found generate a new random embedding and add it to OOV1. V1+OOV1 = V2
        embedding_vector = V1.get_vecs_by_tokens(row['sentences'][i]) # Fetch pretrained embedding vector from GloVe
        a = np.count_nonzero(embedding_vector)
        if a != 0: # Embedding found 
          X_train[index,i,:] = embedding_vector
        elif row['sentences'][i] in OOV1.keys(): # No embedding found in Vocabulary but embedding already generated as OOV token
          X_train[index,i,:] = OOV1[row['sentences'][i]]
        else: # No embedding found neither in Vocabulary nor in OOV1: generate a new embedding and update OOV
          X_train[index,i,:] = np.random.rand((embedding_dim)) # Embedding with random vector
          #X_train[index,i,:] = np.ones((embedding_dim)) # Embedding for fixed UKN placeholder
          OOV1[row['sentences'][i]] = X_train[index,i,:]
        # One Hot Encoding vector fetch and Y_train update
        if (row['tags'][i]) in unique_tags_train:
          ohe_idx = unique_tags_train.index(row['tags'][i]) 
          Y_train[index,i,:] = ohe[ohe_idx]
        else: # No One Hot Encoding vector found
          Y_train[index,i,:] = np.zeros((ohe.shape[1])) # One Hot Encoding for fixed UKN placeholder.
    
    # Validation
    # The model will be validated on X_val and Y_val containing V1+OOV1+OOV2 embeddings (V3) and POS tag one hot encodings found in training (plus placeholder if not found)
    elif ((row['docs'] != "wsj_0100.dp") and (row['docs'].startswith("wsj_010") or row['docs'].startswith("wsj_011") or row['docs'].startswith("wsj_012") or row['docs'].startswith("wsj_013") or row['docs'].startswith("wsj_014") or row['docs'].startswith("wsj_0150"))): 

      if (row['docs'] == "wsj_0101.dp"): # First sentence of validation set 
        OOV2 = OOV1 # initialize OVV2 with embeddings already generate in OOV1
        idx_t = index # offset to start array index from 0
      idx = index - idx_t

      for i in range(0,len(row['sentences'])): # Iterate over the words of the fetched sentence 

        # Embedding from V1+OVV1 (V2), if not found generate a new random embedding and add it to OOV2. V1+OOV1+OOV2 = V3
        embedding_vector = V1.get_vecs_by_tokens(row['sentences'][i]) # Fetch pretrained embedding vector from GloVe
        a = np.count_nonzero(embedding_vector)
        if a != 0: # Embedding found 
          X_val[idx,i,:] = embedding_vector
        elif row['sentences'][i] in OOV2.keys(): # No embedding found in Vocabulary but embedding already generated as OOV2 token
          X_val[idx,i,:] = OOV2[row['sentences'][i]]
        else: # No embedding found neither in Vocabulary nor in OOV1: generate a new embedding and update OOV
          X_val[idx,i,:] = np.random.rand((embedding_dim)) # Embedding with random vector
          OOV2[row['sentences'][i]] = X_val[idx,i,:]
          #X_val[idx,i,:] = np.ones((embedding_dim)) # Embedding for fixed UKN placeholder, it can be changed to random array

        # One Hot Encoding vector fetch and Y_train update
        if (row['tags'][i]) in unique_tags_train:
          ohe_idx = unique_tags_train.index(row['tags'][i]) # always considered vs. training ohe hot encodings
          Y_val[idx,i,:] = ohe[ohe_idx]
        else: # No One Hot Encoding vector found
          Y_val[idx,i,:] = np.zeros((ohe.shape[1])) # One Hot Encoding for fixed UKN placeholder.

    # Test 
    # The model will be tested on X_test and Y_test containing V1+OOV1+OOV2+OOV3 embeddings (V4) and POS tag one hot encodings found in training (plus placeholder if not found)
    elif ((row['docs'] != "wsj_0150.dp") and (row['docs'].startswith("wsj_015") or row['docs'].startswith("wsj_016") or row['docs'].startswith("wsj_017") or row['docs'].startswith("wsj_018") or row['docs'].startswith("wsj_019"))): 

      if (row['docs'] == "wsj_0151.dp"): # First sentence of test set 
        OOV3 = OOV2 # initialize OVV3 with embeddings already generated in OOV2
        idx_v = index # offset to start array index from 0
      idx = index - idx_v

      for i in range(0,len(row['sentences'])): # Iterate over the words of the fetched sentence 

        # Embedding from V1+OVV1+OOV2 (V3), if not found generate a new random embedding and add it to OOV3. V1+OOV1+OOV2+OOV3 = V4
        embedding_vector = V1.get_vecs_by_tokens(row['sentences'][i]) # Fetch pretrained embedding vector from GloVe
        a = np.count_nonzero(embedding_vector)
        if a != 0: # Embedding found 
          X_test[idx,i,:] = embedding_vector
        elif row['sentences'][i] in OOV3.keys(): # No embedding found in Vocabulary but embedding already generated as OOV2 token
          X_test[idx,i,:] = OOV3[row['sentences'][i]]
        else: # No embedding found neither in Vocabulary nor in OOV1: generate a new embedding and update OOV
          X_test[idx,i,:] = np.random.rand((embedding_dim)) # Embedding with random vector
          OOV3[row['sentences'][i]] = X_test[idx,i,:]
          #X_val[idx,i,:] = np.ones((embedding_dim)) # Embedding for fixed UKN placeholder, it can be changed to random array

        # One Hot Encoding vector fetch and Y_train update
        if (row['tags'][i]) in unique_tags_train:
          ohe_idx = unique_tags_train.index(row['tags'][i]) 
          Y_test[idx,i,:] = ohe[ohe_idx]
        else: # No One Hot Encoding vector found
          Y_test[idx,i,:] = np.zeros((ohe.shape[1])) # One Hot Encoding for fixed UKN placeholder.
         


In [19]:
# SB - SOME UNIT CHECKS
# 'Pierre' is an OOV1 word. Its emebedding should be added in training with a Random vector
print(OOV1['Pierre'])

# 'Pierre' is the first word of the first sentence.
print(X_train[0,0])

# Execute notebook more than once and check embedding repeatability.

[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
 0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643
 0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.36636184
 0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041
 0.60754485 0.17052412 0.06505159 0.94888554 0.96563203 0.80839735
 0.30461377 0.09767211 0.68423303 0.44015249 0.12203823 0.49517691
 0.03438852 0.9093204  0.25877998 0.66252228 0.31171108 0.52006802
 0.54671028 0.18485446]
[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
 0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643
 0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.36636184
 0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041
 0.60754485 0.17052412 0.06505159 0.94888554 0.96563203 0.80839735
 0.30461377 0.09767211 0.68423303 0.44

In [23]:
# Vinken is an OOV1 word. File wsj_0001.dp, second word of the first sentence and second word of the second sentence 
print(OOV1['Vinken'])
print(X_train[0,1])
print(X_train[1,1])

[0.96958463 0.77513282 0.93949894 0.89482735 0.59789998 0.92187424
 0.0884925  0.19598286 0.04522729 0.32533033 0.38867729 0.27134903
 0.82873751 0.35675333 0.28093451 0.54269608 0.14092422 0.80219698
 0.07455064 0.98688694 0.77224477 0.19871568 0.00552212 0.81546143
 0.70685734 0.72900717 0.77127035 0.07404465 0.35846573 0.11586906
 0.86310343 0.62329813 0.33089802 0.06355835 0.31098232 0.32518332
 0.72960618 0.63755747 0.88721274 0.47221493 0.11959425 0.71324479
 0.76078505 0.5612772  0.77096718 0.4937956  0.52273283 0.42754102
 0.02541913 0.10789143]
[0.96958463 0.77513282 0.93949894 0.89482735 0.59789998 0.92187424
 0.0884925  0.19598286 0.04522729 0.32533033 0.38867729 0.27134903
 0.82873751 0.35675333 0.28093451 0.54269608 0.14092422 0.80219698
 0.07455064 0.98688694 0.77224477 0.19871568 0.00552212 0.81546143
 0.70685734 0.72900717 0.77127035 0.07404465 0.35846573 0.11586906
 0.86310343 0.62329813 0.33089802 0.06355835 0.31098232 0.32518332
 0.72960618 0.63755747 0.88721274 0.47

In [21]:
# SB More samples. Please note more testing is required.
print(X_val[100,2])
print(Y_val[100,2])
print(X_test[20,10])
print(Y_test[20,10])

[ 0.68046999 -0.039263    0.30186    -0.17792     0.42962     0.032246
 -0.41376001  0.13228001 -0.29846999 -0.085253    0.17117999  0.22419
 -0.10046    -0.43652999  0.33418     0.67846     0.057204   -0.34448001
 -0.42785001 -0.43274999  0.55962998  0.10032     0.18677001 -0.26853999
  0.037334   -2.09319997  0.22171    -0.39868     0.20912001 -0.55725002
  3.88260007  0.47466001 -0.95657998 -0.37788001  0.20869    -0.32752001
  0.12751     0.088359    0.16350999 -0.21634001 -0.094375    0.018324
  0.21048    -0.03088    -0.19722     0.082279   -0.09434    -0.073297
 -0.064699   -0.26043999]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 4.18000013e-01  2.49679998e-01 -4.12420005e-01  1.21699996e-01
  3.45270008e-01 -4.44569997e-02 -4.96879995e-01 -1.78619996e-01
 -6.60229998e-04 -6.56599998e-01  2.78430015e-01 -1.47670001e-01
 -5.56770027e-01  1.46579996e-01 -9.50950012e-03  1.16579998e-0